<a href="https://colab.research.google.com/github/georgehtliu/sentiment-analyzer/blob/master/submission_createcsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries
Joblib used for ease of importing files. Pandas used for data manipulation.

In [8]:
import joblib
import pandas as pd

# Importing Classifier, Vectorizer, and Judgement Data
Support for importing from local storage as well as importing from Google Drive for Google Colab.

In [ ]:
# Assuming files are stored locally in the same directory.
clf_log = joblib.load(SentimentNewton_Log.pkl)

vectorizer = joblib.load(Vectorizer.pkl)

judge_data_path = "judgement_data.csv"

# Uncomment to import files from Google Drive on Google Colab
"""
from google.colab import drive
drive.mount('/content/drive')

clf_path = input('Please enter path to SentimentNewton_Log')
clf_log = joblib.load(clf_path)

vect_path = input('Please enter path to Vectorizer')
vectorizer = joblib.load(vect_path)

judge_data_path = input("Please enter the path to your judgement_data.csv file in your Google Drive.")
"""

# Processing Imported Files
Preparing dataframe and vectors.

In [12]:
# Convert csv file to dataframe
df_judge = pd.read_csv(judge_data_path)
df_mini = df_judge

X = df_mini['Text']
X_vectors= vectorizer.transform(X)

# Writing to CSV File
Edit the *csv_path* variable to decide where the csv will be stored.

In [13]:
csv_path = '/content/drive/My Drive/predicted_labels.csv'

df_mini['Sentiment'] = clf_log.predict(X_vectors)
df_mini.to_csv(csv_path) 
print("Done!")

Done!
